In [1]:
import pandas as pd
from geopy.distance import vincenty
import time

In [2]:
df_old = pd.read_csv(r"G:\work\logistica\distance\input\/1901info_lnglat_gaode.csv")
df_new = pd.read_csv(r"G:\work\logistica\distance\input\/190416Info_lnglat_gaode.csv")

d:\python\envs\jupyterpy36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_old.columns

Index(['所在省', '所在市', '收货地址', '订单编号', '承运商', 'address_all', 're_adcode_gaode',
       're_city_gaode', 're_country_gaode', 're_district_gaode',
       're_formatted_address_gaode', 're_level_gaode', 're_location_gaode',
       're_number_gaode', 're_province_gaode', 're_street_gaode',
       're_township_gaode', 're_siteLng_gaode', 're_siteLat_gaode'],
      dtype='object')

In [30]:
df_new.columns

Index(['所在省', '所在市', '收货人地址', '订单编号', '所在县', 're_adcode_gaode',
       're_city_gaode', 're_country_gaode', 're_district_gaode',
       're_formatted_address_gaode', 're_level_gaode', 're_location_gaode',
       're_number_gaode', 're_province_gaode', 're_street_gaode',
       're_township_gaode', 're_siteLng_gaode', 're_siteLat_gaode'],
      dtype='object')

In [7]:
df_old = df_old[df_old["所在省"].str.contains("所在省") == False]

In [10]:
df_old = df_old[df_old["re_district_gaode"].str.contains("\[\]",regex=True) == False]

In [18]:
df_new = df_new[df_new["re_district_gaode"].str.contains("\[\]",regex=True) == False]


In [22]:
print("df_new:" + str(df_new.shape))
print("df_old:" + str(df_old.shape))

df_new:(2775, 18)
df_old:(521663, 19)


In [28]:
def FenSheng(df_new,df_old,col_province,col_city,col_district,col_lng,col_lat):
    start_time = time.time()
    dfA = pd.DataFrame(columns=['orderID_old','orderID_new','distance'])
    limit_KM = round(3 /(3.14 * 6400 /180),6)
    
    
    province_list = df_new[col_province].unique()
    for sheng in province_list:
#         print(sheng)

        dfold_sheng = df_old[df_old[col_province] == sheng].reset_index(drop=True)
        dfnew_sheng = df_new[df_new[col_province] == sheng].reset_index(drop=True)
        
               
        city_list = dfnew_sheng[col_city].unique()
        for shi in city_list:
#             print(sheng+shi)

            dfold_shi = dfold_sheng[(dfold_sheng[col_city] == shi)].reset_index(drop=True)
            dfnew_shi = dfnew_sheng[(dfnew_sheng[col_city] == shi)].reset_index(drop=True)   
            
            district_list = dfnew_shi[col_district].unique()              
            for xian in district_list:
#                 print(sheng+shi+xian)
#                 print("*"*30)
                dfold = dfold_shi[dfold_shi[col_district] == xian].reset_index(drop=True)
                dfnew = dfnew_shi[dfnew_shi[col_district] == xian].reset_index(drop=True)
                if dfnew.empty == True:
                    print(sheng + xian + "：为空")
                    pass
                elif dfold.empty == True:
                    print("原所在市不存在")
                    pass
                else:
                    for i in range(0,len(dfnew[col_lng])):
                        for j in range(0,len(dfold[col_lng])):
                            start_lng = round(dfnew[col_lng][i],6)
                            start_lat = round(dfnew[col_lat][i],6)
                            end_lng = round(dfold[col_lng][j],6)
                            end_lat = round(dfold[col_lat][j],6)
                            start_lnglat = (start_lat,start_lng)
                            end_lnglat = (end_lat,end_lng)
                            if (abs(start_lat - end_lat) <= limit_KM or abs(start_lng - end_lng) <= limit_KM ):
                                a = vincenty(start_lnglat, end_lnglat).km
                                row={'orderID_old':dfold['订单编号'][j],'orderID_new':dfnew['订单编号'][i],'distance':a}
                                dfA = dfA.append(row,ignore_index=True)
                            else:
                                pass
                    print(str(sheng) + str(shi) + str(xian) + "：已完成")
    end_time = time.time()
    print("总用时：",end_time-start_time)
    return dfA

In [24]:
df_old2 = df_old.sample(n=100000)

In [40]:
dfA = FenSheng(df_new,df_old,"re_province_gaode","re_city_gaode","re_district_gaode","re_siteLng_gaode","re_siteLat_gaode")

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


安徽省安庆市宿松县：已完成
安徽省安庆市怀宁县：已完成
安徽省安庆市桐城市：已完成
安徽省安庆市宜秀区：已完成
安徽省六安市金寨县：已完成
安徽省六安市裕安区：已完成
安徽省六安市舒城县：已完成
安徽省六安市霍邱县：已完成
安徽省六安市金安区：已完成
安徽省合肥市蜀山区：已完成
安徽省合肥市肥西县：已完成
安徽省合肥市瑶海区：已完成
安徽省合肥市包河区：已完成
安徽省合肥市巢湖市：已完成
安徽省合肥市庐阳区：已完成
安徽省合肥市肥东县：已完成
安徽省芜湖市无为县：已完成
安徽省芜湖市弋江区：已完成
安徽省芜湖市芜湖县：已完成
安徽省芜湖市鸠江区：已完成
安徽省芜湖市繁昌县：已完成
安徽省马鞍山市花山区：已完成
安徽省马鞍山市雨山区：已完成
安徽省池州市青阳县：已完成
安徽省滁州市明光市：已完成
安徽省滁州市琅琊区：已完成
安徽省滁州市南谯区：已完成
安徽省宣城市宣州区：已完成
安徽省宣城市宁国市：已完成
安徽省蚌埠市蚌山区：已完成
安徽省蚌埠市怀远县：已完成
安徽省蚌埠市固镇县：已完成
安徽省蚌埠市禹会区：已完成
安徽省阜阳市颍州区：已完成
安徽省阜阳市颍上县：已完成
安徽省阜阳市颍泉区：已完成
安徽省阜阳市临泉县：已完成
安徽省阜阳市颍东区：已完成
安徽省淮南市寿县：已完成
安徽省淮南市凤台县：已完成
安徽省黄山市屯溪区：已完成
安徽省淮北市濉溪县：已完成
安徽省亳州市涡阳县：已完成
安徽省亳州市蒙城县：已完成
北京市北京市房山区：已完成
北京市北京市海淀区：已完成


KeyboardInterrupt: 

In [ ]:
dfB = dfA[dfA['distance'] <= 3]

In [26]:
df_old["re_province_gaode"].unique()

array(['山西省', '福建省', '贵州省', '湖南省', '北京市', '山东省', '甘肃省', '陕西省', '广西壮族自治区',
       '河北省', '西藏自治区', '黑龙江省', '内蒙古自治区', '湖北省', '海南省', '辽宁省', '宁夏回族自治区',
       '河南省', '新疆维吾尔自治区', '安徽省', '青海省', '浙江省', '广东省', '江苏省', '吉林省', '四川省',
       '云南省', '江西省', '上海市', '天津市', '重庆市'], dtype=object)

In [27]:
df_new["re_province_gaode"].unique()

array(['安徽省', '北京市', '福建省', '甘肃省', '广东省', '广西壮族自治区', '贵州省', '海南省', '河北省',
       '河南省', '黑龙江省', '湖北省', '湖南省', '吉林省', '江苏省', '江西省', '辽宁省', '内蒙古自治区',
       '宁夏回族自治区', '青海省', '山东省', '山西省', '陕西省', '上海市', '四川省', '天津市',
       '西藏自治区', '新疆维吾尔自治区', '云南省', '浙江省', '重庆市'], dtype=object)

In [38]:
df_old['re_siteLat_gaode'] = df_old['re_siteLat_gaode'].map(lambda x:float(x))
df_old['re_siteLng_gaode'] = df_old['re_siteLng_gaode'].map(lambda x:float(x))
df_new['re_siteLat_gaode'] = df_new['re_siteLat_gaode'].map(lambda x:float(x))
df_new['re_siteLng_gaode'] = df_new['re_siteLng_gaode'].map(lambda x:float(x))


In [42]:
df_new['re_province_gaode'].value_counts()

广东省         380
河南省         256
山东省         254
江苏省         215
河北省         181
浙江省         139
湖南省         135
辽宁省         129
湖北省         119
陕西省          96
安徽省          82
四川省          79
福建省          77
山西省          74
北京市          65
上海市          62
黑龙江省         56
甘肃省          48
内蒙古自治区       43
云南省          38
吉林省          37
广西壮族自治区      36
江西省          36
新疆维吾尔自治区     34
重庆市          24
天津市          22
贵州省          22
宁夏回族自治区      17
海南省           9
青海省           7
西藏自治区         3
Name: re_province_gaode, dtype: int64